# 噪音学习实验

在通过合理的假设假定芯片的噪音模型之后，通过噪音学习实验获得相关噪音模型的系数是我们对量子芯片、量子门质量的表征手段之一。一方面，这可以帮助我们更好的理解芯片的性质，另一方面，也为各种基于噪音放大或缩小的错误缓解方案（如零噪音外推、概率错误消除等）提供了基础。

噪音学习的基础理论可以参照论文[1]。我们可以假定芯片噪音符合Sparse Pauli-Lindberg模型，在PTM下表达：
$$
\Lambda = \exp \mathcal{L} = \prod_{k\in K} e^{\lambda_k P_k \times P_k^T}e^{-\lambda_k}
$$
其中$K$仅包含单量子比特和相邻比特间的双比特相互作用，$\lambda_k$为噪音系数。我们可以进一步将其化简为：
$$
\Lambda(\rho) = \prod_k [w_k \rho + (1-w_k)(P_k \rho P_k)], w_k = \frac{1+e^{-2\lambda_k}}{2}
$$
通过不同深度的量子线路获取所有泡利保真度，就可以进一步拟合出对应的噪音系数。

因此，基本噪音学习的实验流程可以被如下划分：
1. 根据目标，选取需要学习的量子比特，以及对应的可以同时相互作用的双比特门图样（称之为CZ pattern或CX pattern，接下来以CZ为例）。
2. 生成一组基底，可以覆盖所有的泡利基。对于线性链来说，仅需要9个即可。
3. 生成量子线路：首先进行换基操作，随后生成不同层数的CZ层量子线路，接着对CZ操作Pauli twirl。结果在换回Z基后额外进行测量对角化。
4. 运行量子线路
5. 根据不同层数的测量结果，利用指数拟合获取对应的泡利保真度和测量保真度
6. 利用泡利保真度拟合出噪音系数

下面是要用到的基础工具包，包括计时、文件管理等。

In [ ]:
import time
import networkx as nx
from pathlib import Path
import json
import os

import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

第一步，选取噪音学习所用到的量子比特、双比特门和对应Pattern。所谓Pattern，就是指在考虑串扰情况下同时作用的双门组合，一般根据自己的实际需求决定。这里我们选择了一条链式结构，为12-6-0-1-2-3-4-10-16，共计9个比特，并将CZ门进行奇偶排列组成两个Pattern。注意，这里都是物理比特，且在创建inst_map的时候必须按照从小到大的顺序排布，否则会有bug。

In [ ]:
inst_map = [0,1,2,3,4,6,10,12,16]
coupling_map = [[0,1], [1,2], [2,3],[3,4], [4,10], [10, 16],  [6,0], [12,6]]

pattern1 = [[0,1], [2,3], [4,10], [12,6]]
pattern2 = [[6,0], [1,2], [3,4], [10, 16]]

In [ ]:
cz_patterns = [pattern1, pattern2]

随后应该确定一些基本参数。常见的参数包括线路深度（层数）、线路泡利采样次数等。另外，也要确定真实芯片运行的一系列参数。

首先构造线路的深度列表，我们会用这个值拟合指数曲线获得泡利保真度

In [ ]:
depth = [2, 4, 8, 16]

其次是一些线路设置，包括如何保存线路、Pauli twirl和测量对角化的采样次数等。multi_txt表示是否将不同的量子线路储存在同一个文件中，random_samples代表对单条线路，进行Pauli twirl和readout twirl的线路数量。这里选择50。

In [ ]:
multi_txt = False
random_samples = 1

我们使用本源量子云作为量子芯片后端，为方便调用，已经将qcloud的运行进行了继承。

In [ ]:
from originbench.error_mitigation.algorithm.fit import new_exp

芯片运行的一些基本配置包括：
circuits：待运行的量子线路列表 

key：qcloud的API key，可以在**https://console.originqc.com.cn/zh/computerServices/dashboard**获取，请注意确保自己有足够多的机时 

shots：单条线路的采样次数，一般在噪声学习实验中不必太大，这里设置为256 

auto_mapping：是否开启自动映射。在运行噪音学习时，由于量子比特已经指定，因此应关闭此项。 

circuit_optimize：线路优化，包括将量子门编译为基础量子门等，默认打开。 

measurement_amend：是否开启读取误差矫正。对于噪音学习实验，由于我们要拟合出SPAM噪音，因此该项应该关闭 


In [ ]:
shots = 256
# key = "your api token" # 替换为自己的key

key = (
    'your api token'
)

auto_mapping = False
circuit_optimize = True
measurement_amend = False

还有一些额外的网络参数等，可以根据情况设置。
timeout：线路最大运行时间，超过该时间时会报错，避免无限制运行 

interval：查询间隔，每隔一段时间对芯片进行一次查询 

retry：重复次数，当提交线路或获取结果由于网络原因失败时，会进行一定次数的重试，若超过次数仍然未成功，则终止运行并报错 


In [ ]:
timeout = 1800
interval = 10

使用打包好的噪声学习实验函数，可以自然运行噪音学习流程，该函数会返回噪声学习实验字典结构。如果希望保存噪音学习结果至文件，请将save参数设置为True。

In [ ]:
from originbench.noise_learning.generate_noise_learning import noise_learning

noise_data = noise_learning(inst_map=inst_map,
                            coupling_map=coupling_map,
                            cz_patterns=cz_patterns,
                            key=key, # qcloud后端的参数
                            auto_mapping=auto_mapping,
                            circuit_optimize=circuit_optimize,
                            measurement_amend=measurement_amend,
                            shots=shots,
                            timeout=timeout,
                            interval=interval,
                            save=True)

获取结果的格式：
字典格式，以CZ pattern的字符串格式为key，每个value为一个列表，列表中的每一个元素为一个字典。
该字典包括：
“cz_pattern”：对应的CZ pattern 

“coeffs“：Sparse Pauli-Lindberg模型的噪音系数$\lambda_k$，字典格式，为{pauli:coefficient}的形式，如{'IIX':0.001, ...} 

”fidelity“：不同泡利基下拟合出来的单层量子门保真度，字典格式 

”spam_noise“：不同泡利基下拟合出来的，由线路初始化和测量带来的误差，字典格式。

分析拟合结果，画图。注意，由于noise coefficient的数目比较多，一般图会很长。注意调节图的大小。


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

noise_data_pattern_1 = list(noise_data.values())[0][0]
cz_pattern = noise_data_pattern_1['cz_pattern']
fidelity = noise_data_pattern_1['fidelity']
coeffs = noise_data_pattern_1['coeffs']
print(fidelity)
print(coeffs)


In [ ]:
from matplotlib import use

plt.figure(figsize=(len(coeffs), 8))
plt.bar(coeffs.keys(), coeffs.values())
plt.xlabel('pauli')
plt.title(f'noise coefficient of cz pattern {cz_pattern}')
plt.show()

# 参考文献
[1] Berg, E. van den, Minev, Z. K., Kandala, A. & Temme, K. Probabilistic error cancellation with sparse Pauli-Lindblad models on noisy quantum processors. Preprint at http://arxiv.org/abs/2201.09866 (2022).